# BLU10 - Learning Notebook - Part 3 of 3 - Non-personalized recommenders

In [1]:
import os
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
import pandas as pd

### Table of contents
[1. Non-personalized RS](#1.-Non-personalized-RS)   
[2. Loading the data](#2.-Loading-the-data)   
&emsp;[1.1 Community matrix](#1.1-Community-matrix)   
[3. Building the ratings matrix](#3.-Building-the-ratings-matrix)   
&emsp;[3.1 Building it with NumPy](#3.1-Building-it-with-NumPy)   
&emsp;[3.2 Building it with Pandas](#3.2-Building-it-with-Pandas)   
[4. Density and sparsity scores](#4.-Density-and-sparsity-scores)   
&emsp;[4.1 Density](#4.1-Density)   
&emsp;[4.2 Sparsity](#4.2-Sparsity)   
[5. Aggregated opinion](#5.-Aggregated-opinion)   
&emsp;[5.1 Most-rated](#5.1-Most-rated)   
&emsp;[5.2 Yeah, but what if most ratings are negative?](#5.2-Yeah,-but-what-if-most-ratings-are-negative?)   
[6. Powering up with summary statistics](#6.-Powering-up-with-summary-statistics)   
[7. Association rules](#7.-Association-rules)   
&emsp;[7.1 Support](#7.1-Support)   
&emsp;[7.2 Apriori](#7.2-Apriori)   
&emsp;[7.3 Confidence](#7.3-Confidence)   
&emsp;[7.4 Lift](#7.4-Lift)   
&emsp;[7.5 Association rules](#7.5-Association-rules)   

It is finally time to implement our first basic recommender system!

# 1. Non-personalized RS

The core functions of any RS is to identify useful items for the user.

Going back to our framework, non-personalized RS typically include the base model (users, items, and ratings).

![Recommender Sytems Framework](./media/recommender_systems_framework_base.png)

*Fig.1 - RS framework with a community and the basic model.*

We consider users, however, only as providers of ratings, ignoring user preferences shown by the ratings they give. **The rationale is that a generic user also likes something that is liked by many users.**

This approach is particularly relevant in the absence of information about the user preferences, like for users that didn't provide any rating or if we simply did not collect any user information (user profile). If we are unable to predict the utility of an item for a particular user, then we recommend an item with high utility for many users. 

We will use the non-personalized algorithm example here to give you an idea of building an RS and to train your NumPy skills.

# 2. Loading the data

First, we read the data into Python and NumPy.

In [2]:
def read_data():
    
    path = os.path.join('data', 'ml-latest-small', 'ratings.csv')
    data = np.genfromtxt(path, delimiter=',',skip_header=1, usecols=[0, 1, 2])
    return data


data = read_data()
pd.DataFrame(data).head(5)

,0,1,2
0,1.0,31.0,2.5
1,1.0,1029.0,3.0
2,1.0,1061.0,3.0
3,1.0,1129.0,2.0
4,1.0,1172.0,4.0


What we have here is:
* User identification (userId) in the first column
* Item identification (movieId) in the second column
* Rating in the third column.

The dataset also contains a timestamp that we don't need at the moment. We are not using the column names to make further array manipulation easier.

We have 100004 datapoints - ratings of an item by a user: 

In [3]:
data.shape[0]

100004

**We'll work with this data in array form throughout this notebook.**

For more information, explore the `../data/ml-latest-small/` folder.

# 3. Building the ratings matrix

The second step is to transform this representation into a ratings matrix, with:
* Users as rows
* Items as columns
* Ratings as values.

We use the unique values for users and items, storing the indices that can be used to reconstruct the original array.

Then, we create a matrix, all filled with zeros, the size we want:
* The number of unique users is the number of rows
* The number of unique items is the number of columns.

Finally, we fill in the rating values, using the stored indexes, in a vectorized way.

## 3.1 Building it with Numpy

In [4]:
def make_ratings(data):
    
    users, user_pos = np.unique(data[:, 0], return_inverse=True)
    items, item_pos = np.unique(data[:, 1], return_inverse=True)
    
    R = np.zeros((len(users), len(items)))
    R[user_pos, item_pos] = data[:, 2]
    
    return R


R = make_ratings(data)
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

Take your time to read through and experiment with the code as you go.

**How many values do you think will be non zero in this matrix?**

In [5]:
len(R[R>0])

100004

Exactly the number of rows we had in our tabular format!

## 3.2 Building it with Pandas 

Another beautiful constructor that we can use is the Pandas pivot function (that you already met when learning about tidy data). It can be convenient to retain indexes for our products and users in a data frame instead of having a numpy array and Pandas rescues us on that.

You have to use the pivot method on a dataframe. It takes as arguments: 
- index: The row index (normally the User)
- columns: The column indexes (normally the Product)
- values: The values of the matrix (normally the Ratings)

In [6]:
pd.DataFrame(data).pivot(index=0, columns=1, values=2).head(5)

1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,161084.0,161155.0,161594.0,161830.0,161918.0,161944.0,162376.0,162542.0,162672.0,163949.0
0,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Nice! We can power this up with fillna()!

In [7]:
pd.DataFrame(data).pivot(index=0, columns=1, values=2).fillna(0).head(5)

1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,161084.0,161155.0,161594.0,161830.0,161918.0,161944.0,162376.0,162542.0,162672.0,163949.0
0,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A thing of beauty. One-line of code! But remember that with great power comes great responsability - this matrix is really hungry and eats a lot of space!

# 4. Density and sparsity scores

Now we compute the density and the sparsity of the ratings matrix.

## 4.1 Density 

Density is the fraction of the non-zero elements of the matrix. We will use the array method `nonzero` to return a mask of the elements that are non-zero - this is another way to get the elements that are not zero in a matrix - if you had negative elements it would be more efficient to use this method instead of doing R[R>0].

In [8]:
R.nonzero()

(array([  0,   0,   0, ..., 670, 670, 670]),
 array([  30,  833,  859, ..., 4597, 4610, 4696]))

We can compute the density score, as: 

$$Density = \frac{|R'|}{|R|}$$

Where $|R'|$ is equal to the elements that are not zero in the matrix.

In [9]:
R[R.nonzero()].size / R.size

0.016439141608663475

Holy moly, at least now we know what we are up against! - Only 1.6% of the matrix has values that are not zero.

## 4.2 Sparsity

Sparsity is the opposite of density - the fraction of elements that are zero in a matrix! Simply put:

$$Sparsity = 1- \frac{|R'|}{|R|}$$

In [10]:
1 - R[R.nonzero()].size / R.size

0.9835608583913366

These two attributes complement each other and they are important when dealing with rating matrixes. In this case: 
- **Our ratings matrix is ~2% dense and ~98% sparse!**

# 5. Aggregated opinion

Again, the most important aspect of non-personalized recommenders is that we predict the utility of an item for all the users.

Perhaps the oldest RS is aggregated opinion, i.e., most popular/most hated (Think Billboard or [IMDb Bottom 100](https://www.imdb.com/chart/bottom)). Another simple option is the most rated items. Most rated items tend to be those most interacted with by the users or those that elicit the strongest opinions.

## 5.1 Most-rated

According to popular opinion, the most popular items are the ones with most ratings.

We start by checking which elements are greater than zero.

In [11]:
def is_rating(R):
    return np.greater(R, 0)

is_rating(R)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

Recall that each row corresponds to user and each column to an item, so we can sum the results in each column to know how many real ratings exist for that item.

In [12]:
def count_ratings(R):
    R_ = is_rating(R)
    return R_.sum(axis=0)

count_ratings(R)

array([247, 107,  59, ...,   1,   1,   1])

Now, we can have a function that retrieves the top-N most-rated items.

In [13]:
def most_rated(R, n):
    R_ = count_ratings(R)
    return np.negative(R_).argsort()[:n]


most_rated(R, 3)

array([321, 266, 284])

## 5.2 Yeah, but what if most ratings are negative?

We can extend the function above to mimic another popular algorithm, "Highest % of Top Ratings".

Let's say a positive rating is anything above the value of 3 (e.g., 3 stars).

In [14]:
def count_positive_ratings(R, threshold):
    R_ = is_above_threshold(R, threshold)
    return R_.sum(axis=0)


def is_above_threshold(R, threshold):
    return np.greater(R, threshold)


count_positive_ratings(R, 3)

array([182,  51,  24, ...,   1,   0,   1])

Now, we just need to count the number of positive ratings and sort the resulting array.

In [15]:
def most_positive_ratings(R, threshold, n):
    R_ = count_positive_ratings(R, threshold)
    return np.negative(R_).argsort()[:n]


most_positive_ratings(R, 3, 3)

array([284, 321, 266])

# 6. Powering up with summary statistics

Until now we have only used counts as measures of popularity: count the number of ratings and count the number of positive ratings. But we can rely on good old statistics to help us out here.

Probably the most popular non-personalized algorithm is the average rating.

Popularized at first by Amazon and Ebay and then IMDB and Netflix, this is a basic yet widely used algorithm.

The first step is to remove the zeros, so that they don't affect our average.

In [16]:
def remove_zeros(R):
    R_ = R.copy()
    R_[R_ == 0] = np.NaN
    
    return R_


remove_zeros(R)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [ 5., nan, nan, ..., nan, nan, nan]])

And now, we can safely compute the average rating per item and sort the array. We are using the `np.nanmean` function which ignores the NaNs when computing the mean.

In [17]:
R.shape

(671, 9066)

In [18]:
def mean_ratings(R):
    R_ = remove_zeros(R)
    return np.nanmean(R_, axis=0)


mean_ratings(R)

array([3.87246964, 3.40186916, 3.16101695, ..., 5.        , 3.        ,
       5.        ])

In [19]:
def best_mean_rating(R, n):
    R_ = mean_ratings(R)
    return np.negative(R_).argsort()[:n]


best_mean_rating(R, 3)

array([9065, 8119, 8125])

There are alternatives, such as computing the "mean rating for users that liked this item", that we won't explore here.

It's also increasingly popular to show a histogram alongside mean ratings, to give a sense of the distribution of ratings. Or to normalize the mean by the number of ratings so that items with a low number of ratings do not get an advantage.

# 7. Association rules

Perhaps one of the most interesting (and also very popular) non-personalized algorithms is "people that buy X, also buy Y".

These kinds of algorithms are called association rules. Here, for the sake of conciseness, we use `mlxtend` to implement some of them. You can find the documentation [here](http://rasbt.github.io/mlxtend/).

(Yes, we are cheating. We should be implementing it with NumPy, but it's easier with mlxtend!)

Before looking at association rules, we need to define a helper concept, support, and look at different metrics to measure the strength of the association rules.

## 7.1 Support

Support measures the frequency of an itemset in a database of transactions. Let's say we have a database of purchases in a supermarket - a table where rows are purchases and columns are items. The support of a set of purchased items is the fraction of purchases that contains that specific item set. For instance, in a pandemic, everybody buys toilet paper - the support of toilet paper is 1 because it is contained in every purchase done in the supermarket. If every tenth purchase contains toilet paper and bananas, the support of the item set {toilet paper, bananas} is 0.1.

Now onto the association rule.

## 7.2 Apriori

Apriori is a function used to identify common item sets, i.e., stuff that usually goes together. Usually, we are interested only in item sets that are frequent, i.e. that have some minimal support (occurence threshold). This is is how the algorithm goes:

* We identify individual items that satisfy a minimum occurrence threshold (minimum support)
* Then we extend the item sets, adding one item at a time
* Every time we check if the resulting item set satisfies the specified threshold
* The algorithm stops when there are no more items to add that meet the threshold.

Clearly, if a certain set meets the minimum occurence threshold, all its subsets also meet it.

For our database of purchases (rows are purchase, columns are items), `mlxtend` expects a one-hot input, i.e., 0/1 or True/False to indicate which items belong to which purchase. The `min_support` parameter indicates the minimum occurrence threshold (fraction of purchases containing the given item set).

Unfortunately, `mlxtend` only supports dataframes at this point. The items in the item sets are returned as respective column indices.

In [20]:
def get_apriori_itemsets(R, min_support=0.3):
    R_ = pd.DataFrame(R > 0)
    return apriori(R_, min_support)


get_apriori_itemsets(R)

,support,itemsets
0,0.368107,(0)
1,0.339791,(100)
2,0.433681,(232)
3,0.482861,(266)
4,0.463487,(284)
5,0.508197,(321)
6,0.317437,(406)
7,0.408346,(427)
8,0.363636,(472)
9,0.320417,(521)


After identiying the item sets with the `apriori` function, we can use the `association_rules` function to find out how strong is the association between the items in the item sets.

The strength of the association rules are measured with different metrics, we will look at two of them. Notice that association rules can have a direction - we can be interested only in implications. Do people who buy toilet paper also buy bananas? Do people who buy bananas also buy toilet paper? That's why talk about antecedent and consequent items or item sets.

## 7.3 Confidence

Given two item sets $i$, called antecendent, and $j$, called consequent, the confidence refers to how frequently the item set $j$ is purchased, given that item set $i$ was purchased (how many people who buy toilet paper also buy bananas?):

$$Confidence\{i \to j \} = \frac{Support\{i, j\}}{Support\{i\}}$$

Or, in a more familiar way, confidence is the conditional probability of $j$ given $i$:

$$P(j|i) = \frac{P(i \cap j)}{P(i)}$$

However, do $i$ and $j$ occur for the same users for a reason, or is it random? What if $j$ is a trendy item?

## 7.4 Lift

Meet the toilet paper trap: just because people buy toilet paper all the time, it doesn't mean toilet paper goes well with bananas.

Fortunately, there is a better way. 

The lift algorithm, which takes into consideration the popularity of the items and normalizes the confidence to the support of $j$.

$$Lift\{i \to j\}\ =\ \frac{Confidence\{i \to j \}}{Support\{j\}}\ =\ \frac{Support\{i, j\}}{Support\{i\} * Support\{j\}}$$

The denominator is the likelihood that $i$ and $j$ appear together by chance, so lift questions whether $i$ makes $j$ more probable or not. Think of lift as a metric that is able to take into account the actual popularity of the item.

Mlxtend implements two more metrics, leverage and conviction, measuring the independence of the item sets. Check out the definitions [here](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#metrics).


## 7.5 Association rules
Let's look at the association rules between two-item item sets (antecendent and consequent). Each rows is one association rule whose strength is indicated by different metrics:

In [21]:
def get_rules(R, min_support=.3, min_threshold=1.2):
    itemsets = get_apriori_itemsets(R, min_support=0.3)
    return association_rules(itemsets, metric="lift", min_threshold=min_threshold)

get_rules(R)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(232),(953),0.433681,0.348733,0.302534,0.697595,2.000367,0.151295,2.153621
1,(953),(232),0.348733,0.433681,0.302534,0.867521,2.000367,0.151295,4.274794
2,(266),(284),0.482861,0.463487,0.326379,0.675926,1.458348,0.102578,1.655525
3,(284),(266),0.463487,0.482861,0.326379,0.704180,1.458348,0.102578,1.748153
4,(321),(266),0.508197,0.482861,0.344262,0.677419,1.402927,0.098874,1.603130
5,(266),(321),0.482861,0.508197,0.344262,0.712963,1.402927,0.098874,1.713379
6,(266),(525),0.482861,0.453055,0.333830,0.691358,1.525991,0.115067,1.772101
7,(525),(266),0.453055,0.482861,0.333830,0.736842,1.525991,0.115067,1.965127
8,(321),(284),0.508197,0.463487,0.321908,0.633431,1.366663,0.086365,1.463607
9,(284),(321),0.463487,0.508197,0.321908,0.694534,1.366663,0.086365,1.610009


Wrapping up: 
- Non personalized recommenders do not take into account specific user preferences or characteristics.
- Non personalized recommenders approaches are the simplest way to design recommendation engines.
- It's really important to know how to handle matrix sparsity as it will impact your workflow until the end.

Now, we have the foundations to tackle more complex recommendation approaches, coming up in the next BLUs.

Time to practice!